### Importando pacotes e dataset

In [173]:
import os

import matplotlib.pyplot as plt
# import earthpy as et
import numpy as np

import pandas as pd

df = pd.read_csv('83377.csv')

df.head()

,Unnamed: 0,Estacao,Data,Hora,Precipitacao,TempBulboSeco,TempBulboUmido,TempMaxima,TempMinima,UmidadeRelativa,PressaoAtmEstacao,PressaoAtmMar,DirecaoVento,VelocidadeVento,Insolacao,Nebulosidade,Evaporacao Piche,Temp Comp Media,Umidade Relativa Media,Velocidade do Vento Media
0,12132121,83377,21/08/1961,0,NaN,22.6,14.1,27.9,NaN,36.0,901.5,NaN,NaN,NaN,10.0,NaN,NaN,22.14,36.25,1.366667
1,12132122,83377,21/08/1961,1200,NaN,23.8,15.2,NaN,13.8,37.0,889.2,NaN,9.0,1.5,NaN,0.0,NaN,NaN,NaN,NaN
2,12132123,83377,21/08/1961,1800,NaN,24.2,15.4,NaN,NaN,36.0,901.8,NaN,14.0,2.6,NaN,7.0,NaN,NaN,NaN,NaN
3,12132124,83377,22/08/1961,0,NaN,22.6,14.1,29.7,NaN,36.0,901.5,NaN,0.0,0.0,9.4,5.0,NaN,23.52,28.25,1.033333
4,12132125,83377,22/08/1961,1200,0.0,24.8,14.6,NaN,13.9,30.0,888.3,NaN,14.0,2.1,NaN,4.0,NaN,NaN,NaN,NaN


In [174]:
df.columns

Index(['Unnamed: 0', 'Estacao', 'Data', 'Hora', 'Precipitacao',
       'TempBulboSeco', 'TempBulboUmido', 'TempMaxima', 'TempMinima',
       'UmidadeRelativa', 'PressaoAtmEstacao', 'PressaoAtmMar', 'DirecaoVento',
       'VelocidadeVento', 'Insolacao', 'Nebulosidade', 'Evaporacao Piche',
       'Temp Comp Media', 'Umidade Relativa Media',
       'Velocidade do Vento Media'],
      dtype='object')

In [175]:
def derive_nth_day_feature(df, feature, N):
    rows = df.shape[0]
    nth_prior_measurements = [None] * N + [df[feature][i - N] for i in range(N, rows)]
    col_name = "{}_{}".format(feature, N)
    df[col_name] = nth_prior_measurements


def dataframe_sort(df):
    # make list of original features without meantempm, mintempm, and maxtempm
    to_remove = [feature
                 for feature in df
                 if feature not in ['Precipitacao', 'TempBulboSeco', 'TempBulboUmido', 'TempMaxima', 'TempMinima',
                                    'UmidadeRelativa', 'PressaoAtmEstacao', 'PressaoAtmMar', 'DirecaoVento',
                                    'VelocidadeVento', 'Insolacao', 'Nebulosidade', 'Evaporacao Piche',
                                    'Temp Comp Media', 'Umidade Relativa Media', 'Velocidade do Vento Media']]

    # make a list of columns to keep
    to_keep = [col for col in df.columns if col not in to_remove]

    # select only the columns in to_keep and assign to df
    df = df[to_keep]

    for feature in df:
        if feature != 'Data':
            for N in range(1, 4):
                derive_nth_day_feature(df, feature, N)

    return df

In [176]:
# df = dataframe_sort(df)
# df.head()


In [177]:
df = df.drop(columns=['Unnamed: 0', 'Estacao', 'Velocidade do Vento Media',
                      'Umidade Relativa Media', 'Temp Comp Media', 'Evaporacao Piche', 'Insolacao',
                      'TempMaxima'], axis=1)
df = df[df['Precipitacao'].notna()]
df = df.reset_index(drop=True)
df.head()
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21017 entries, 0 to 21016
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Data               21017 non-null  object 
 1   Hora               21017 non-null  int64  
 2   Precipitacao       21017 non-null  float64
 3   TempBulboSeco      20903 non-null  float64
 4   TempBulboUmido     20191 non-null  float64
 5   TempMinima         20994 non-null  float64
 6   UmidadeRelativa    21001 non-null  float64
 7   PressaoAtmEstacao  20896 non-null  float64
 8   PressaoAtmMar      6652 non-null   float64
 9   DirecaoVento       21000 non-null  float64
 10  VelocidadeVento    21001 non-null  float64
 11  Nebulosidade       21002 non-null  float64
dtypes: float64(10), int64(1), object(1)
memory usage: 1.9+ MB


In [178]:
df.head()


,Data,Hora,Precipitacao,TempBulboSeco,TempBulboUmido,TempMinima,UmidadeRelativa,PressaoAtmEstacao,PressaoAtmMar,DirecaoVento,VelocidadeVento,Nebulosidade
0,22/08/1961,1200,0.0,24.8,14.6,13.9,30.0,888.3,NaN,14.0,2.1,4.0
1,23/08/1961,1200,1.0,19.8,14.6,13.8,56.0,888.3,NaN,14.0,2.6,5.0
2,24/08/1961,1200,0.0,22.4,15.8,13.2,49.0,902.0,NaN,5.0,2.6,7.0
3,25/08/1961,1200,0.0,23.2,15.0,12.9,40.0,902.2,NaN,14.0,2.6,0.0
4,26/08/1961,1200,0.0,25.6,15.2,12.8,30.0,901.9,NaN,14.0,1.5,2.0


In [179]:
X=df[['Hora', 'TempBulboSeco', 'TempBulboUmido', 'TempMinima', 'UmidadeRelativa', 'PressaoAtmEstacao', 'PressaoAtmMar', 'DirecaoVento', 'VelocidadeVento', 'Nebulosidade']]
y= df.apply(lambda row: 1 if row.Precipitacao > 1 else 0, axis=1)


In [180]:
from sklearn.model_selection import train_test_split

xtrain,xtest,ytrain,ytest=train_test_split(X,y,test_size=0.2, random_state=41)


In [181]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
xtrain = scaler.fit_transform(xtrain)
xtest = scaler.transform(xtest)

In [182]:
from keras.layers import Dense, Dropout
from keras.models import Sequential

ann  = Sequential()
ann.add(Dense(units= 32,kernel_initializer= 'uniform', activation = 'relu', input_dim=10))
ann.add(Dropout(0.2))
ann.add(Dense(units= 16,kernel_initializer= 'uniform', activation = 'relu'))
ann.add(Dropout(0.2))
ann.add(Dense(units= 1,kernel_initializer= 'uniform', activation = 'sigmoid'))
ann.compile(optimizer='adam',
              loss='mean_squared_error',
              metrics=['accuracy'])

ann.fit(xtrain, ytrain, batch_size=10, epochs=10,verbose= 1)

Epoch 1/10
1682/1682 [==============================] - 1s 390us/step - loss: nan - accuracy: 0.7055

In [183]:
Y_pred = ann.predict(xtest)
# Y_pred = [ 1 if y>=0.2 else 0 for y in Y_pred]
print(Y_pred)


[[nan]
 [nan]
 [nan]
 ...
 [nan]
 [nan]
 [nan]]


In [184]:
# from sklearn.metrics import confusion_matrix
# cm = confusion_matrix(ytest, Y_pred)
# print(cm)

